In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os
import requests
import pandas

from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import HTML
from example_helpers import *

MDSTUDIO_URL = 'http://mdstudio-smartcyp.dev.openrisknet.org/'
FAME_URL = 'http://fame3.dev.openrisknet.org'
METPRED_URL = 'http://metpred.prod.openrisknet.org/v2'

data = get_dataset()

## SOM prediction comparing Docking, SMARTCyp and FAME

In [ ]:
selection = ('sitaxentan', 'flucloxacillin', 'pioglitazone')
for case_name in selection:
    
    case_data = data[case_name]
    
    # Get SMARTCyp prediction
    response = requests.post('{0}/som_prediction'.format(MDSTUDIO_URL),
                             files={'ligand_file': case_data['mol2']}, data={'cyp': '3A4'}) 
    response_df = pandas.DataFrame.from_dict(response.json(), orient='index')
    
    # Get FAME3 prediction
    response = requests.post('{0}/predictFromFiles'.format(FAME_URL), files={'files': case_data['mol']},
                             data={'includeMDL': False}, headers={"accept" : "application/json"})
    response_df, famecutoff = process_fame_results(response.json(), response_df)
    
    # Get MetPred prediction
    response = requests.get('{0}/prediction'.format(METPRED_URL), params={'compound': case_data['mol']})
    response_df, metpredcutoff = process_metpred_results(response.json(), response_df)
    
    # Draw molecule
    response_df_prob = response_df[['Docking', 'SMARTCyp', 'Fame', 'MetPred', 'MetPred reaction']].fillna(0)
    rdmol = Chem.rdmolfiles.MolFromMol2File('{0}.mol2'.format(case_name), sanitize=False)
    dock_svg = show_predictions(rdmol, response_df_prob['Docking'])
    smart_svg = show_predictions(rdmol, response_df_prob['SMARTCyp'])
    fame_svg = show_predictions(rdmol, response_df_prob['Fame'], cutoff=famecutoff)
    metpred_svg = show_predictions(rdmol, response_df_prob['MetPred'], cutoff=metpredcutoff)
    
    # Display Pandas Dataframe and 2D depictions
    display(HTML('<h3 align="center">{0}</h3>'.format(case_name)))
    display(HTML(no_wrap_div.format(dock_svg, smart_svg, fame_svg, metpred_svg)))
    display(style_dataframe(response_df_prob, fame_cutoff=famecutoff))